In [2]:
from numpy.lib.format import open_memmap
_scores = open_memmap('/home/jinxulin/MISS/Classification/trak_results/scores/cifar10.mmap')
print(_scores.shape)

(50000, 10000)


In [3]:
import wget
from pathlib import Path

masks_url = 'https://www.dropbox.com/s/x76uyen8ffkjfke/mask.npy?dl=1'
margins_url = 'https://www.dropbox.com/s/q1dxoxw78ct7c27/val_margins.npy?dl=1'

masks_path = Path('.').joinpath('mask.npy')
wget.download(masks_url, out=str(masks_path), bar=None)


margins_path = Path('.').joinpath('val_margins.npy')
wget.download(margins_url, out=str(margins_path), bar=None)

'val_margins.npy'

In [4]:
import numpy as np
import torch
from scipy.stats import spearmanr
from tqdm import tqdm

masks = torch.as_tensor(np.load(masks_path, mmap_mode='r')).float()
margins = torch.as_tensor(np.load(margins_path, mmap_mode='r'))

val_inds = np.arange(10_000)
preds = masks @ _scores
rs = []
ps = []
for ind, j in tqdm(enumerate(val_inds)):
    r, p = spearmanr(preds[:, ind], margins[:, j])
    rs.append(r)
    ps.append(p)
rs, ps = np.array(rs), np.array(ps)
print(f'Correlation: {rs.mean():.3f} (avg p value {ps.mean():.6f})')


/tmp/ipykernel_392079/1948449808.py:6: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1699449229234/work/torch/csrc/utils/tensor_numpy.cpp:206.)
  masks = torch.as_tensor(np.load(masks_path, mmap_mode='r')).float()
10000it [00:07, 1317.30it/s]

Correlation: 0.114 (avg p value 0.000774)
